In [1]:
import cv2
import numpy as np
import os
from imutils import face_utils
import imutils
import dlib
import time
from math import ceil
from PIL import Image
from keras.applications.vgg16 import VGG16
import tkinter as tk
from tkinter import font as tkfont
from tkinter import messagebox,PhotoImage
import serial

In [2]:
face_classifier = cv2.CascadeClassifier(r'./models/haarcascade_frontalface_default.xml')
eye_th = 0.3
frames_for_blink = 2
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'./models/shape_predictor_68_face_landmarks.dat')
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
vgg = VGG16(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)
th = 0.93
data = {}
fake_ctr = 1
num_samples = 50

In [9]:
serial_port = 'COM3'
arduino=serial.Serial(serial_port,baudrate=9600, bytesize=8, parity=serial.PARITY_EVEN, stopbits=1)
arduino.timeout = 1

def arduino_on():
    arduino.write(b'1')

        
def arduino_off():
    arduino.write(b'0')
    
arduino_off()

In [3]:
def preprocess(img):
    
    img = img/255
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3)
    pred = vgg.predict(img).flatten()
    pred = (pred - pred.mean()) / pred.std()
    return pred

def similarity(a, b):
    
    num = np.dot(a, b)
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    return num/denom

def train(name):
    
    path = r'./Images/' + name
    val = np.zeros(25088)                           #25088 is size after flattening vgg output
    for img in os.listdir(path):
        
        img_path = path + '\\' + img
        img = cv2.imread(img_path)
        pred = preprocess(img)
        val += pred
        
    data[name] = val / num_samples
    
    return data[name]

def predict(img):
    
    pred = preprocess(img)
    max_sim = 0
    per = ""
    for name in data.keys():
        
        sim = similarity(pred, data[name])
        if sim > max_sim:
            max_sim = sim
            per = name
    
    if max_sim < th:
        return 0, "nomatch"
    
    return max_sim, per

In [4]:
def eye_aspect_ratio(eye):
    
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def face_extractor(img):

        faces = face_classifier.detectMultiScale(img, 1.3, 5)
        if faces is ():
            return None

        for (x,y,w,h) in faces:
            x=x-10
            y=y-10
            cropped_face = img[y:y+h+50, x:x+w+50]

        return cropped_face

def test_cam():
    
    cap = cv2.VideoCapture(0)
    t_end = time.time() + 60 * (10/60)
    
    while time.time() < t_end:
        
        ret, frame = cap.read()
        tt = t_end - time.time()
        cv2.putText(frame, str(ceil(tt)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.putText(frame, "PRESS ENTER KEY TO EXIT", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) == 13:
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
    
def add_face(name) :
    
    path = r'./Images/' + name
    os.mkdir(path)

    cap = cv2.VideoCapture(0)
    count = 0

    while True:

        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (400, 400))

            file_name_path = path + '\\' + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Capture', face)


        if cv2.waitKey(1) == 13 or count == num_samples: 
            break

    cap.release()
    cv2.destroyAllWindows()      
    print("Collecting Samples Complete")
    
    train(name)
    print("Face Successfully Added")
    
def blink_detect():
    
    cap = cv2.VideoCapture(0)
    blinks = 0
    ctr = 0
    frames = []
    t_end = time.time() + 60 * (5/60)

    while time.time() < t_end:

        ret, frame = cap.read()     
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        
        for rect in rects:
        
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            if ear < eye_th:
                ctr += 1

            else:

                if ctr >= frames_for_blink:
                    blinks += 1
                ctr = 0
                
            tt = t_end - time.time()
            cv2.putText(frame, str(ceil(tt)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
       
        cv2.imshow("Frame", frame)
        
        if face_extractor(frame) is not None:
            
            face = cv2.resize(face_extractor(frame), (224, 224))
            cv2.imshow("face", face)
            frames.append(face)
        
        if cv2.waitKey(1) == 13: 
                break
        
    
    cap.release()
    cv2.destroyAllWindows()
    
    return np.asarray(frames), blinks

def check_face():
    
    frames, blinks = blink_detect()  
    test = frames.mean(axis = 0)
    if blinks == 0:
        return 0, "intruder"
           
    else:
        print("Facial Recognition in Progress....")
        sim, name = predict(test)
        print(sim)
        if sim == 0:
            PIL_image = Image.fromarray(np.uint8(test)).convert('RGB')
            global fake_ctr
            PIL_image.save(r'./Intruder/' + str(fake_ctr) + '.jpg')
            fake_ctr += 1
        
        return sim, name

In [5]:
class App(tk.Tk):

    def __init__(self):
        
        tk.Tk.__init__(self)
        self.title_font = tkfont.Font(family='Helvetica', size=16, weight="bold")
        self.title("Face Recognizer")
        self.resizable(False, False)
        self.geometry("500x250")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.active_name = None
        container = tk.Frame(self)
        container.grid(sticky="nsew")
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        self.frames = {}
        for F in (StartPage, PageOne, PageTwo, PageThree, PageFour):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.frames[page_name] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame("StartPage")

    def show_frame(self, page_name):
            frame = self.frames[page_name]
            frame.tkraise()

    def on_closing(self):

        if messagebox.askokcancel("Quit", "Are you sure?"):
            self.destroy()


class StartPage(tk.Frame):

        def __init__(self, parent, controller):
            tk.Frame.__init__(self, parent)
            self.controller = controller
            label = tk.Label(self, text="        Home Page        ", font=self.controller.title_font,fg="#263942")
            label.grid(row=0, sticky="ew")
            button1 = tk.Button(self, text="    Add a User    ", fg="#ffffff", bg="#263942",command=lambda: self.controller.show_frame("PageOne"))
            button2 = tk.Button(self, text="Face Recognition", fg="#ffffff", bg="#263942",command=lambda: self.controller.show_frame("PageTwo"))
            button3 = tk.Button(self, text="  Quit  ", fg="#263942", bg="#ffffff", command=self.on_closing)
            button1.grid(row=1, column=0, ipady=3, ipadx=7)
            button2.grid(row=3, column=0, ipady=3, ipadx=2)
            button3.grid(row=5, column=0, ipady=3, ipadx=32)


        def on_closing(self):
            if messagebox.askokcancel("Quit", "Are you sure?"):
                self.controller.destroy()


class PageOne(tk.Frame):
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        tk.Label(self, text="Enter the name", fg="#263942", font='Helvetica 12 bold').grid(row=0, column=0, pady=10, padx=5)
        self.user_name = tk.Entry(self, borderwidth=3, bg="lightgrey", font='Helvetica 11')
        self.user_name.grid(row=0, column=1, pady=10, padx=10)
        self.buttontest = tk.Button(self, text="Test Camera", fg="#ffffff", bg="#263942", command=self.test)
        self.buttoncanc = tk.Button(self, text="Cancel", bg="#ffffff", fg="#263942", command=lambda: controller.show_frame("StartPage"))
        self.buttonext = tk.Button(self, text="Next", fg="#ffffff", bg="#263942", command=self.start_training)
        self.buttoncanc.grid(row=1, column=0, pady=10, ipadx=5, ipady=4)
        self.buttonext.grid(row=1, column=2, pady=10, ipadx=5, ipady=4)
        self.buttontest.grid(row=1, column=1, pady=10, ipadx=5, ipady=4)

    
    def test(self):
        test_cam()
        
    def start_training(self):
        
        if self.user_name.get() == "None":
            messagebox.showerror("Error", "Name cannot be 'None'")
            return

        elif len(self.user_name.get()) == 0:
            messagebox.showerror("Error", "Name cannot be empty!")
            return
        
        name = self.user_name.get()
        self.controller.active_name = name
        self.controller.show_frame("PageThree")


class PageTwo(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        tk.Label(self, text="Please blink a few times while image is being captured", fg="#263942", font='Helvetica 12 bold').grid(row=0, column=0, padx=10, pady=10)
        self.buttonext = tk.Button(self, text="Next", command= self.nextfoo, fg="#ffffff", bg="#263942")
        self.buttonext.grid(row=1, ipadx=5, ipady=4, column=1, pady=10)

    def nextfoo(self):
        self.controller.show_frame("PageFour")


class PageThree(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller        
        self.capturebutton = tk.Button(self, text="Start Capturing", fg="#ffffff", bg="#263942", command=self.capimg)
        self.buttoncanc = tk.Button(self, text="Cancel", bg="#ffffff", fg="#263942", command=lambda: controller.show_frame("StartPage"))
        self.buttoncanc.grid(row=1, column=2, pady=10, ipadx=5, ipady=4)
        self.capturebutton.grid(row=1, column=0, ipadx=5, ipady=4, padx=10, pady=20)

    def capimg(self):
        add_face(self.controller.active_name)
        messagebox.showinfo("RESULT", "FACE ADDED")
        self.controller.show_frame("PageFour")
        
class PageFour(tk.Frame):

    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        self.controller = controller
        label = tk.Label(self, text="Face Recognition", font='Helvetica 16 bold')
        label.grid(row=0,column=0, sticky="ew")
        button1 = tk.Button(self, text="Face Recognition", command=self.openwebcam, fg="#ffffff", bg="#263942")
        button2 = tk.Button(self, text="Go to Home Page", command=lambda: self.controller.show_frame("StartPage"), bg="#ffffff", fg="#263942")
        button1.grid(row=1,column=0, sticky="ew", ipadx=5, ipady=4, padx=10, pady=10)
        button2.grid(row=1,column=1, sticky="ew", ipadx=5, ipady=4, padx=10, pady=10)

    def openwebcam(self):
        
        sim, name = check_face()
        if sim == 0:
            if name == "nomatch" :
                messagebox.showinfo("RESULT", "FACE NOT RECOGNISED.")
            else:
                messagebox.showinfo("RESULT", "Plese show valid face")
            
        else:
#             arduino_on()
            messagebox.showinfo("RESULT", "WELCOME {}".format(name))
            time.sleep(5)
#             arduino_off()        

In [6]:
app = App()
app.mainloop()

Face Successfully Added
